<a href="https://github.com/jbutle55/detectron2/blob/master/Run_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/jbutle55/yolov3-tf2
%cd yolov3-tf2/
!pip install -r requirements-gpu.txt

##### 2.  Check Tensorflow2 version

In [ ]:
import tensorflow as tf
tf.__version__

import os
import cv2
import logging
import time
from yolov3_tf2.models import YoloV3
from yolov3_tf2.utils import draw_outputs
from yolov3_tf2.dataset import load_tfrecord_dataset, transform_images

from google.colab.patches import cv2_imshow


##### Load weights

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights
!python convert.py


In [ ]:
class_path = '/content/yolov3-tf2/data/coco.names'  # Path to classes file
weights = '/content/yolov3-tf2/checkpoints/yolov3.tf'  # Path to weight file
size = 608  # Resize images to size - 416 04 608
video = '/content/drive/MyDrive/TestVideoData/MFOV - EOW.avi'  # Path to input image
tfrecord = None  # tfrecord instead of image or None
output = '/content/yolov3-tf2/data/output.avi'  # Path to output image
num_classes = 80  # Number of classes in model
model_path = 'trained.h5'
loading_model = False
output_format = 'XVID'

physical_devices = tf.config.experimental.list_physical_devices('GPU')
for physical_device in physical_devices:
    tf.config.experimental.set_memory_growth(physical_device, True)

if loading_model is False:
    yolo = YoloV3(classes=num_classes)
    yolo.load_weights(weights).expect_partial()
    print('weights loaded')
else:
    yolo = load_model(model_path)

class_names = [c.strip() for c in open(class_path).readlines()]
print('classes loaded')

In [ ]:
times = []

try:
    vid = cv2.VideoCapture(int(video))
except:
    vid = cv2.VideoCapture(video)
out = None
if output:
    # by default VideoCapture returns float instead of int
    width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    codec = cv2.VideoWriter_fourcc(*output_format)
    out = cv2.VideoWriter(output, codec, fps, (width, height))


In [ ]:
from google.colab.patches import cv2_imshow

while True:
    _, img = vid.read()
    if img is None:
        logging.warning("Empty Frame")
        time.sleep(0.1)
        continue
    img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_in = tf.expand_dims(img_in, 0)
    img_in = transform_images(img_in, size)
    t1 = time.time()
    boxes, scores, classes, nums = yolo.predict(img_in)
    t2 = time.time()
    times.append(t2-t1)
    times = times[-20:]
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)
    img = cv2.putText(img, "Time: {:.2f}ms".format(sum(times)/len(times)*1000), (0, 30),
                      cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
    if output:
        out.write(img)
    # cv2_imshow(img)
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {'name': name, 'mimeType': 'application/octet-stream'}
  media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)
  created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

  return created

save_file_to_drive('output.avi', output)
